In [63]:
import os
import sys
from glob import glob
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from tqdm.notebook import tqdm
from time import time
import pickle
import math

import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing as mp

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, models
from torchvision.transforms import Resize, ToTensor, Normalize

import warnings
warnings.filterwarnings('ignore')

In [64]:
import random

# Set random seed
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)  # type: ignore
torch.backends.cudnn.deterministic = True  # type: ignore
torch.backends.cudnn.benchmark = True  # type: ignore

# Prepare Data

In [83]:
class cfg:
    data_dir = '/opt/ml/input/data/train'
    img_dir = f'{data_dir}/images'
    df_path = f'{data_dir}/train.csv'

In [84]:
# 데이터프레임 -> 데이터셋
class CustomDatasetFromDF(Dataset):
    def __init__(self, DataFrame):
        
        # Transforms
        self.to_tensor = transforms.Compose([transforms.Resize(256),
                                             transforms.CenterCrop(224),
                                             transforms.ToTensor()])
        self.data_info = DataFrame
        
        # First column contains the image paths
        self.image_arr = np.asarray(self.data_info.loc[:, 'imageFile'])
        
        # Second column is the labels
        self.label_arr = np.asarray(self.data_info.loc[:, 'allcate'])
       
        # Calculate len
        self.data_len = len(self.data_info.index)

    def __getitem__(self, index):
        # Get image name from the pandas df
        single_image_name = self.image_arr[index]
        
        # Open image
        img_as_img = Image.open(single_image_name)
        
        # Transform image to tensor
        img_as_tensor = self.to_tensor(img_as_img)

        # Get label(class) of the image based on the cropped pandas column
        single_image_label = self.label_arr[index]

        return (img_as_tensor, single_image_label)

    def __len__(self):
        return self.data_len

In [85]:
with open(f'{cfg.data_dir}/df_0825.pickle', 'rb') as f:
    df = pickle.load(f)

In [86]:
df.head()

,id,race,path,gender,age,classMask,"G, A, M",allcate,maskState,imageFile
0,000001,Asian,000001_female_Asian_45,0,1,0,"(0, 1, 0)",10,incorrect,/opt/ml/input/data/train/images/000001_female_...
1,000001,Asian,000001_female_Asian_45,0,1,1,"(0, 1, 1)",4,mask,/opt/ml/input/data/train/images/000001_female_...
2,000001,Asian,000001_female_Asian_45,0,1,1,"(0, 1, 1)",4,mask,/opt/ml/input/data/train/images/000001_female_...
3,000001,Asian,000001_female_Asian_45,0,1,1,"(0, 1, 1)",4,mask,/opt/ml/input/data/train/images/000001_female_...
4,000001,Asian,000001_female_Asian_45,0,1,1,"(0, 1, 1)",4,mask,/opt/ml/input/data/train/images/000001_female_...


In [87]:
# 5 - fold cross validation
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = []
for train_idx, valid_idx in skf.split(df, df['allcate']):
    folds.append((train_idx, valid_idx))

for train_idx, valid_idx in folds:
    df_train, df_valid = df.iloc[train_idx], df.iloc[valid_idx]

In [88]:
train_data = CustomDatasetFromDF(df.iloc[folds[0][0]])
train_loader = DataLoader(train_data, shuffle=True, batch_size=1)
next(iter(train_loader))

[tensor([[[[0.8667, 0.8667, 0.8667,  ..., 0.8549, 0.8549, 0.8549],
           [0.8667, 0.8667, 0.8667,  ..., 0.8549, 0.8549, 0.8549],
           [0.8667, 0.8667, 0.8667,  ..., 0.8549, 0.8549, 0.8549],
           ...,
           [0.7569, 0.7529, 0.7451,  ..., 0.6980, 0.6980, 0.7020],
           [0.7451, 0.7412, 0.7373,  ..., 0.6941, 0.6980, 0.6980],
           [0.7373, 0.7333, 0.7255,  ..., 0.6902, 0.6941, 0.6941]],
 
          [[0.8824, 0.8824, 0.8824,  ..., 0.8706, 0.8706, 0.8706],
           [0.8824, 0.8824, 0.8824,  ..., 0.8706, 0.8706, 0.8706],
           [0.8824, 0.8824, 0.8824,  ..., 0.8706, 0.8706, 0.8706],
           ...,
           [0.1451, 0.1412, 0.1333,  ..., 0.1255, 0.1255, 0.1294],
           [0.1333, 0.1294, 0.1255,  ..., 0.1216, 0.1255, 0.1255],
           [0.1255, 0.1216, 0.1137,  ..., 0.1176, 0.1216, 0.1216]],
 
          [[0.8863, 0.8863, 0.8863,  ..., 0.8745, 0.8745, 0.8745],
           [0.8863, 0.8863, 0.8863,  ..., 0.8745, 0.8745, 0.8745],
           [0.8863, 0.88

In [89]:
resnet50 = models.resnet50(pretrained=True)
print("네트워크 필요 입력 채널 개수", resnet50.conv1.weight.shape[1])
print("네트워크 출력 채널 개수 (예측 class type 개수)", resnet50.fc.weight.shape[0])

네트워크 필요 입력 채널 개수 3
네트워크 출력 채널 개수 (예측 class type 개수) 1000


In [90]:
resnet50.fc.in_features

2048

In [91]:
target_model = models.resnet50(pretrained=True)
CLASS_NUM = 18
target_model.fc = torch.nn.Linear(in_features=2048, out_features=CLASS_NUM, bias=True)
nn.init.xavier_uniform_(target_model.fc.weight)
stdv =  1 / math.sqrt(target_model.fc.in_features)
target_model.fc.bias.data.uniform_(-stdv, stdv)

print("네트워크 필요 입력 채널 개수", target_model.conv1.weight.shape[1])
print("네트워크 출력 채널 개수 (예측 class type 개수)", target_model.fc.weight.shape[0])

네트워크 필요 입력 채널 개수 3
네트워크 출력 채널 개수 (예측 class type 개수) 18


In [92]:
train_data = CustomDatasetFromDF(df.iloc[folds[0][0]])
valid_data = CustomDatasetFromDF(df.iloc[folds[0][1]])

# Mnist Dataset을 DataLoader에 붙이기
BATCH_SIZE = 64
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [93]:
## 2. mnist train 데이터 셋을 resnet50 모델에 학습하기

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

target_model.to(device) # Resnent 18 네트워크의 Tensor들을 GPU에 올릴지 Memory에 올릴지 결정함

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 20 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

dataloaders = {
    "train" : train_dataloader,
    "test" : valid_dataloader
}

best_models = {}

cuda is using!


In [76]:
### 학습 코드 시작
best_test_accuracy = 0.
best_test_loss = 0.


for epoch in range(NUM_EPOCH):
  for phase in ["train", "test"]:
    running_loss = 0.
    running_acc = 0.
    if phase == "train":
      target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "test":
      target_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
      # hint - with, pbar
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = target_model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

      running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)

    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
      best_test_accuracy = epoch_acc
    if phase == "test" and best_test_loss < epoch_loss: # phase가 test일 때, best loss 계산
      best_test_loss = epoch_loss
      best_model = target_model
    
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")


현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.377, 평균 Accuracy : 0.881



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.239, 평균 Accuracy : 0.910



현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.076, 평균 Accuracy : 0.975



현재 epoch-1의 test-데이터 셋에서 평균 Loss : 0.129, 평균 Accuracy : 0.957



현재 epoch-2의 train-데이터 셋에서 평균 Loss : 0.046, 평균 Accuracy : 0.986



현재 epoch-2의 test-데이터 셋에서 평균 Loss : 0.104, 평균 Accuracy : 0.967



현재 epoch-3의 train-데이터 셋에서 평균 Loss : 0.036, 평균 Accuracy : 0.988



현재 epoch-3의 test-데이터 셋에서 평균 Loss : 0.080, 평균 Accuracy : 0.976



현재 epoch-4의 train-데이터 셋에서 평균 Loss : 0.018, 평균 Accuracy : 0.995



현재 epoch-4의 test-데이터 셋에서 평균 Loss : 0.060, 평균 Accuracy : 0.978
학습 종료!
최고 accuracy : 0.9780423641204834, 최고 낮은 loss : 0.23931330035209025


## 프리즈 실험

In [104]:
# resnet34, epoch20
target_model = models.resnet34(pretrained=True)
CLASS_NUM = 18
target_model.fc = torch.nn.Linear(in_features=512, out_features=CLASS_NUM, bias=True)
nn.init.xavier_uniform_(target_model.fc.weight)
stdv =  1 / math.sqrt(target_model.fc.in_features)
target_model.fc.bias.data.uniform_(-stdv, stdv)

train_data = CustomDatasetFromDF(df.iloc[folds[0][0]])
valid_data = CustomDatasetFromDF(df.iloc[folds[0][1]])

# Mnist Dataset을 DataLoader에 붙이기
BATCH_SIZE = 64
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

target_model.to(device) # Resnent 18 네트워크의 Tensor들을 GPU에 올릴지 Memory에 올릴지 결정함

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 20 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

dataloaders = {
    "train" : train_dataloader,
    "test" : valid_dataloader
}

# freeze
for param in target_model.parameters():
    param.requires_grad = False

CLASS_NUM = 18

# melt layer4
for param in target_model.layer4.parameters():
    param.requires_grad = True

# melt fc
for param in target_model.fc.parameters():
    param.requires_grad = True

### 학습 코드 시작
best_test_accuracy = 0.
best_test_loss = 0.

for epoch in range(NUM_EPOCH):
  for phase in ["train", "test"]:
    running_loss = 0.
    running_acc = 0.
    if phase == "train":
      target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "test":
      target_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
      # hint - with, pbar
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = target_model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

      running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)

    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
      best_test_accuracy = epoch_acc
    if phase == "test" and best_test_loss < epoch_loss: # phase가 test일 때, best loss 계산
      best_test_loss = epoch_loss
      best_model = target_model
    
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

cuda is using!



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.514, 평균 Accuracy : 0.838



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.249, 평균 Accuracy : 0.911



현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.103, 평균 Accuracy : 0.968



현재 epoch-1의 test-데이터 셋에서 평균 Loss : 0.170, 평균 Accuracy : 0.944


...


현재 epoch-5의 train-데이터 셋에서 평균 Loss : 0.014, 평균 Accuracy : 0.996



현재 epoch-5의 test-데이터 셋에서 평균 Loss : 0.135, 평균 Accuracy : 0.955



현재 epoch-6의 train-데이터 셋에서 평균 Loss : 0.028, 평균 Accuracy : 0.991



현재 epoch-6의 test-데이터 셋에서 평균 Loss : 0.140, 평균 Accuracy : 0.955


KeyboardInterrupt: 

In [ ]:
# resnet34, epoch20
target_model = models.resnet34(pretrained=True)
CLASS_NUM = 18
target_model.fc = torch.nn.Linear(in_features=512, out_features=CLASS_NUM, bias=True)
nn.init.xavier_uniform_(target_model.fc.weight)
stdv =  1 / math.sqrt(target_model.fc.in_features)
target_model.fc.bias.data.uniform_(-stdv, stdv)

train_data = CustomDatasetFromDF(df.iloc[folds[0][0]])
valid_data = CustomDatasetFromDF(df.iloc[folds[0][1]])

# Mnist Dataset을 DataLoader에 붙이기
BATCH_SIZE = 64
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

target_model.to(device) # Resnent 18 네트워크의 Tensor들을 GPU에 올릴지 Memory에 올릴지 결정함

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 20 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

dataloaders = {
    "train" : train_dataloader,
    "test" : valid_dataloader
}

# freeze
for param in target_model.parameters():
    param.requires_grad = False

CLASS_NUM = 18

# melt layer4
for param in target_model.layer4.parameters():
    param.requires_grad = True

# melt fc
for param in target_model.fc.parameters():
    param.requires_grad = True

### 학습 코드 시작
best_test_accuracy = 0.
best_test_loss = 0.

for epoch in range(NUM_EPOCH):
  for phase in ["train", "test"]:
    running_loss = 0.
    running_acc = 0.
    if phase == "train":
      target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "test":
      target_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
      # hint - with, pbar
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = target_model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

      running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)

    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
      best_test_accuracy = epoch_acc
    if phase == "test" and best_test_loss < epoch_loss: # phase가 test일 때, best loss 계산
      best_test_loss = epoch_loss
      best_model = target_model
    
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

# fold 별 모델 학습

In [106]:
# resnet152, epoch 10
target_model = models.resnet152(pretrained=True)
CLASS_NUM = 18
target_model.fc = torch.nn.Linear(in_features=2048, out_features=CLASS_NUM, bias=True)
nn.init.xavier_uniform_(target_model.fc.weight)
stdv =  1 / math.sqrt(target_model.fc.in_features)
target_model.fc.bias.data.uniform_(-stdv, stdv)

train_data = CustomDatasetFromDF(df.iloc[folds[0][0]])
valid_data = CustomDatasetFromDF(df.iloc[folds[0][1]])

# Mnist Dataset을 DataLoader에 붙이기
BATCH_SIZE = 64
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

target_model.to(device) # Resnent 18 네트워크의 Tensor들을 GPU에 올릴지 Memory에 올릴지 결정함

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 10 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

dataloaders = {
    "train" : train_dataloader,
    "test" : valid_dataloader
}

# freeze
for param in target_model.parameters():
    param.requires_grad = False

CLASS_NUM = 18

# melt layer4
for param in target_model.layer4.parameters():
    param.requires_grad = True

# melt fc
for param in target_model.fc.parameters():
    param.requires_grad = True

### 학습 코드 시작
best_test_accuracy = 0.
best_test_loss = 0.

for epoch in range(NUM_EPOCH):
  for phase in ["train", "test"]:
    running_loss = 0.
    running_acc = 0.
    if phase == "train":
      target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "test":
      target_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
      # hint - with, pbar
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = target_model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

      running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)

    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
      best_test_accuracy = epoch_acc
    if phase == "test" and best_test_loss < epoch_loss: # phase가 test일 때, best loss 계산
      best_test_loss = epoch_loss
      best_model = target_model
    
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

cuda is using!



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.443, 평균 Accuracy : 0.860



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.195, 평균 Accuracy : 0.929



현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.082, 평균 Accuracy : 0.974



현재 epoch-1의 test-데이터 셋에서 평균 Loss : 0.106, 평균 Accuracy : 0.963


...


현재 epoch-7의 test-데이터 셋에서 평균 Loss : 0.100, 평균 Accuracy : 0.971



현재 epoch-8의 train-데이터 셋에서 평균 Loss : 0.008, 평균 Accuracy : 0.998



현재 epoch-8의 test-데이터 셋에서 평균 Loss : 0.070, 평균 Accuracy : 0.981



현재 epoch-9의 train-데이터 셋에서 평균 Loss : 0.007, 평균 Accuracy : 0.998



현재 epoch-9의 test-데이터 셋에서 평균 Loss : 0.068, 평균 Accuracy : 0.979
학습 종료!
최고 accuracy : 0.9814814925193787, 최고 낮은 loss : 0.1951458761330556


In [95]:
## 5-폴드 결과 합쳐보기

In [119]:
# 1.

target_model = models.resnet50(pretrained=True)
CLASS_NUM = 18
target_model.fc = torch.nn.Linear(in_features=2048, out_features=CLASS_NUM, bias=True)
nn.init.xavier_uniform_(target_model.fc.weight)
stdv =  1 / math.sqrt(target_model.fc.in_features)
target_model.fc.bias.data.uniform_(-stdv, stdv)

train_data = CustomDatasetFromDF(df.iloc[folds[0][0]])
valid_data = CustomDatasetFromDF(df.iloc[folds[0][1]])

# Mnist Dataset을 DataLoader에 붙이기
BATCH_SIZE = 64
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

## 2. mnist train 데이터 셋을 resnet50 모델에 학습하기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

target_model.to(device) # Resnent 18 네트워크의 Tensor들을 GPU에 올릴지 Memory에 올릴지 결정함

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 15 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

dataloaders = {
    "train" : train_dataloader,
    "test" : valid_dataloader
}

# freeze
for param in target_model.parameters():
    param.requires_grad = False

CLASS_NUM = 18

# melt layer4
for param in target_model.layer4.parameters():
    param.requires_grad = True

# melt fc
for param in target_model.fc.parameters():
    param.requires_grad = True

### 학습 코드 시작
best_test_accuracy = 0.
best_test_loss = 1.

for epoch in range(NUM_EPOCH):
  for phase in ["train", "test"]:
    running_loss = 0.
    running_acc = 0.
    if phase == "train":
      target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "test":
      target_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
      # hint - with, pbar
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = target_model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

      running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)
    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
      best_test_accuracy = epoch_acc
    if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
      print('Get best model!')
      best_test_loss = epoch_loss
      best_model_1 = target_model
    
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

cuda is using!



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.427, 평균 Accuracy : 0.866



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.204, 평균 Accuracy : 0.933
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.079, 평균 Accuracy : 0.975



현재 epoch-1의 test-데이터 셋에서 평균 Loss : 0.115, 평균 Accuracy : 0.962
Get best model!


...


현재 epoch-12의 test-데이터 셋에서 평균 Loss : 0.083, 평균 Accuracy : 0.981



현재 epoch-13의 train-데이터 셋에서 평균 Loss : 0.006, 평균 Accuracy : 0.999



현재 epoch-13의 test-데이터 셋에서 평균 Loss : 0.092, 평균 Accuracy : 0.978



현재 epoch-14의 train-데이터 셋에서 평균 Loss : 0.002, 평균 Accuracy : 1.000



현재 epoch-14의 test-데이터 셋에서 평균 Loss : 0.063, 평균 Accuracy : 0.983
Get best model!
학습 종료!
최고 accuracy : 0.9833333492279053, 최고 낮은 loss : 0.0634587104444065


In [120]:
# 2.

target_model = models.resnet50(pretrained=True)
CLASS_NUM = 18
target_model.fc = torch.nn.Linear(in_features=2048, out_features=CLASS_NUM, bias=True)
nn.init.xavier_uniform_(target_model.fc.weight)
stdv =  1 / math.sqrt(target_model.fc.in_features)
target_model.fc.bias.data.uniform_(-stdv, stdv)

train_data = CustomDatasetFromDF(df.iloc[folds[1][0]])
valid_data = CustomDatasetFromDF(df.iloc[folds[1][1]])

# Mnist Dataset을 DataLoader에 붙이기
BATCH_SIZE = 64
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

## 2. mnist train 데이터 셋을 resnet50 모델에 학습하기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

target_model.to(device) # Resnent 18 네트워크의 Tensor들을 GPU에 올릴지 Memory에 올릴지 결정함

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 15 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

dataloaders = {
    "train" : train_dataloader,
    "test" : valid_dataloader
}

# freeze
for param in target_model.parameters():
    param.requires_grad = False

CLASS_NUM = 18

# melt layer4
for param in target_model.layer4.parameters():
    param.requires_grad = True

# melt fc
for param in target_model.fc.parameters():
    param.requires_grad = True

### 학습 코드 시작
best_test_accuracy = 0.
best_test_loss = 1.

for epoch in range(NUM_EPOCH):
  for phase in ["train", "test"]:
    running_loss = 0.
    running_acc = 0.
    if phase == "train":
      target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "test":
      target_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
      # hint - with, pbar
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = target_model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

      running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)
    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
      best_test_accuracy = epoch_acc
    if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
      print('Get best model!')
      best_test_loss = epoch_loss
      best_model_2 = target_model
    
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

cuda is using!



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.431, 평균 Accuracy : 0.864



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.159, 평균 Accuracy : 0.947
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.079, 평균 Accuracy : 0.975



현재 epoch-1의 test-데이터 셋에서 평균 Loss : 0.101, 평균 Accuracy : 0.967
Get best model!


...


현재 epoch-12의 test-데이터 셋에서 평균 Loss : 0.078, 평균 Accuracy : 0.980



현재 epoch-13의 train-데이터 셋에서 평균 Loss : 0.015, 평균 Accuracy : 0.995



현재 epoch-13의 test-데이터 셋에서 평균 Loss : 0.070, 평균 Accuracy : 0.979



현재 epoch-14의 train-데이터 셋에서 평균 Loss : 0.007, 평균 Accuracy : 0.998



현재 epoch-14의 test-데이터 셋에서 평균 Loss : 0.076, 평균 Accuracy : 0.983
학습 종료!
최고 accuracy : 0.9838624596595764, 최고 낮은 loss : 0.06209983522720803


In [121]:
# 3.

target_model = models.resnet50(pretrained=True)
CLASS_NUM = 18
target_model.fc = torch.nn.Linear(in_features=2048, out_features=CLASS_NUM, bias=True)
nn.init.xavier_uniform_(target_model.fc.weight)
stdv =  1 / math.sqrt(target_model.fc.in_features)
target_model.fc.bias.data.uniform_(-stdv, stdv)

train_data = CustomDatasetFromDF(df.iloc[folds[2][0]])
valid_data = CustomDatasetFromDF(df.iloc[folds[2][1]])

# Mnist Dataset을 DataLoader에 붙이기
BATCH_SIZE = 64
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

## 2. mnist train 데이터 셋을 resnet50 모델에 학습하기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

target_model.to(device) # Resnent 18 네트워크의 Tensor들을 GPU에 올릴지 Memory에 올릴지 결정함

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 15 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

dataloaders = {
    "train" : train_dataloader,
    "test" : valid_dataloader
}

# freeze
for param in target_model.parameters():
    param.requires_grad = False

CLASS_NUM = 18

# melt layer4
for param in target_model.layer4.parameters():
    param.requires_grad = True

# melt fc
for param in target_model.fc.parameters():
    param.requires_grad = True

### 학습 코드 시작
best_test_accuracy = 0.
best_test_loss = 1.

for epoch in range(NUM_EPOCH):
  for phase in ["train", "test"]:
    running_loss = 0.
    running_acc = 0.
    if phase == "train":
      target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "test":
      target_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
      # hint - with, pbar
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = target_model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

      running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)
    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
      best_test_accuracy = epoch_acc
    if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
      print('Get best model!')
      best_test_loss = epoch_loss
      best_model_3 = target_model
    
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

cuda is using!



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.433, 평균 Accuracy : 0.859



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.175, 평균 Accuracy : 0.941
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.074, 평균 Accuracy : 0.977



현재 epoch-1의 test-데이터 셋에서 평균 Loss : 0.135, 평균 Accuracy : 0.955
Get best model!


...


현재 epoch-12의 test-데이터 셋에서 평균 Loss : 0.085, 평균 Accuracy : 0.977



현재 epoch-13의 train-데이터 셋에서 평균 Loss : 0.005, 평균 Accuracy : 0.999



현재 epoch-13의 test-데이터 셋에서 평균 Loss : 0.059, 평균 Accuracy : 0.985
Get best model!



현재 epoch-14의 train-데이터 셋에서 평균 Loss : 0.007, 평균 Accuracy : 0.998



현재 epoch-14의 test-데이터 셋에서 평균 Loss : 0.093, 평균 Accuracy : 0.975
학습 종료!
최고 accuracy : 0.9851852059364319, 최고 낮은 loss : 0.059035697943366515


In [122]:
# 4.

target_model = models.resnet50(pretrained=True)
CLASS_NUM = 18
target_model.fc = torch.nn.Linear(in_features=2048, out_features=CLASS_NUM, bias=True)
nn.init.xavier_uniform_(target_model.fc.weight)
stdv =  1 / math.sqrt(target_model.fc.in_features)
target_model.fc.bias.data.uniform_(-stdv, stdv)

train_data = CustomDatasetFromDF(df.iloc[folds[3][0]])
valid_data = CustomDatasetFromDF(df.iloc[folds[3][1]])

# Mnist Dataset을 DataLoader에 붙이기
BATCH_SIZE = 64
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

## 2. mnist train 데이터 셋을 resnet50 모델에 학습하기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

target_model.to(device) # Resnent 18 네트워크의 Tensor들을 GPU에 올릴지 Memory에 올릴지 결정함

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 15 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

dataloaders = {
    "train" : train_dataloader,
    "test" : valid_dataloader
}

# freeze
for param in target_model.parameters():
    param.requires_grad = False

CLASS_NUM = 18

# melt layer4
for param in target_model.layer4.parameters():
    param.requires_grad = True

# melt fc
for param in target_model.fc.parameters():
    param.requires_grad = True

### 학습 코드 시작
best_test_accuracy = 0.
best_test_loss = 1.

for epoch in range(NUM_EPOCH):
  for phase in ["train", "test"]:
    running_loss = 0.
    running_acc = 0.
    if phase == "train":
      target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "test":
      target_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
      # hint - with, pbar
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = target_model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

      running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)
    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
      best_test_accuracy = epoch_acc
    if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
      print('Get best model!')
      best_test_loss = epoch_loss
      best_model_4 = target_model
    
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

cuda is using!



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.432, 평균 Accuracy : 0.861



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.172, 평균 Accuracy : 0.942
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.077, 평균 Accuracy : 0.978



현재 epoch-1의 test-데이터 셋에서 평균 Loss : 0.114, 평균 Accuracy : 0.962
Get best model!


...


현재 epoch-12의 test-데이터 셋에서 평균 Loss : 0.094, 평균 Accuracy : 0.982



현재 epoch-13의 train-데이터 셋에서 평균 Loss : 0.010, 평균 Accuracy : 0.997



현재 epoch-13의 test-데이터 셋에서 평균 Loss : 0.243, 평균 Accuracy : 0.952



현재 epoch-14의 train-데이터 셋에서 평균 Loss : 0.018, 평균 Accuracy : 0.995



현재 epoch-14의 test-데이터 셋에서 평균 Loss : 0.078, 평균 Accuracy : 0.979
Get best model!
학습 종료!
최고 accuracy : 0.9820106029510498, 최고 낮은 loss : 0.07784833696295253


In [123]:
# 5.

target_model = models.resnet50(pretrained=True)
CLASS_NUM = 18
target_model.fc = torch.nn.Linear(in_features=2048, out_features=CLASS_NUM, bias=True)
nn.init.xavier_uniform_(target_model.fc.weight)
stdv =  1 / math.sqrt(target_model.fc.in_features)
target_model.fc.bias.data.uniform_(-stdv, stdv)

train_data = CustomDatasetFromDF(df.iloc[folds[4][0]])
valid_data = CustomDatasetFromDF(df.iloc[folds[4][1]])

# Mnist Dataset을 DataLoader에 붙이기
BATCH_SIZE = 64
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

## 2. mnist train 데이터 셋을 resnet50 모델에 학습하기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

target_model.to(device) # Resnent 18 네트워크의 Tensor들을 GPU에 올릴지 Memory에 올릴지 결정함

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 15 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.Adam(target_model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

dataloaders = {
    "train" : train_dataloader,
    "test" : valid_dataloader
}

# freeze
for param in target_model.parameters():
    param.requires_grad = False

CLASS_NUM = 18

# melt layer4
for param in target_model.layer4.parameters():
    param.requires_grad = True

# melt fc
for param in target_model.fc.parameters():
    param.requires_grad = True

### 학습 코드 시작
best_test_accuracy = 0.
best_test_loss = 1.

for epoch in range(NUM_EPOCH):
  for phase in ["train", "test"]:
    running_loss = 0.
    running_acc = 0.
    if phase == "train":
      target_model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "test":
      target_model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
      # hint - with, pbar
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = target_model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

      running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)
    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
      best_test_accuracy = epoch_acc
    if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
      print('Get best model!')
      best_test_loss = epoch_loss
      best_model_5 = target_model
    
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

cuda is using!



현재 epoch-0의 train-데이터 셋에서 평균 Loss : 0.423, 평균 Accuracy : 0.865



현재 epoch-0의 test-데이터 셋에서 평균 Loss : 0.196, 평균 Accuracy : 0.931
Get best model!



현재 epoch-1의 train-데이터 셋에서 평균 Loss : 0.078, 평균 Accuracy : 0.975



현재 epoch-1의 test-데이터 셋에서 평균 Loss : 0.146, 평균 Accuracy : 0.952
Get best model!


...


현재 epoch-12의 test-데이터 셋에서 평균 Loss : 0.076, 평균 Accuracy : 0.980



현재 epoch-13의 train-데이터 셋에서 평균 Loss : 0.003, 평균 Accuracy : 0.999



현재 epoch-13의 test-데이터 셋에서 평균 Loss : 0.055, 평균 Accuracy : 0.985
Get best model!



현재 epoch-14의 train-데이터 셋에서 평균 Loss : 0.018, 평균 Accuracy : 0.995



현재 epoch-14의 test-데이터 셋에서 평균 Loss : 0.163, 평균 Accuracy : 0.958
학습 종료!
최고 accuracy : 0.9846560955047607, 최고 낮은 loss : 0.05476845637633194


# 제출

In [107]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

In [108]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transforms.Compose([transforms.Resize(256),
                                             transforms.CenterCrop(224),
                                             transforms.ToTensor()])

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [196]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
models = [best_model_1, best_model_2, best_model_3, best_model_4, best_model_5]

all_predictions = []

for model in tqdm(models):
    model.to(device)
    model.eval()
    
    pred_list = []
    # 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
    for images in tqdm(loader):
        with torch.no_grad():
            images = images.to(device)
            pred = model(images)
            pred_list.extend(pred.cpu().numpy())
    all_predictions.append(np.array(pred_list))

# 각 fold로 학습시킨 모델의 결과 soft voting
total_pred = (all_predictions[0] + all_predictions[1] + all_predictions[2] + all_predictions[3] + all_predictions[4]).argmax(-1)
submission['ans'] = total_pred

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission_50_5fold_20210826.csv'), index=False)
print('test inference is done!')

ValueError: Length of values (5) does not match length of index (12600)

In [198]:
submission

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,13
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,1
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,13
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,13
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,12
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,1
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,4
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,9
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,1
